In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *

# project sources

## dropped/excluded

* US 50 (withdrawn)
* US 101 Marin (minimal accessibility/speed benefits, worthwhile project better analyzed in another context)
* MST reroute (apparent marginal/negative benefits, institutional barriers to relocating/adding service)

# read and aggregate

In [33]:
first_set = gpd.read_file('./june16_corridor_metrics_speeds.geojson')
first_set = first_set >> filter(-_.corridor.str.contains('us50')) #dropped
first_set['submission_round'] = 1

In [49]:
(first_set >> select(-_.geometry)).to_csv('pending_accessibility.csv')

In [34]:
second_set = gpd.read_file('./sep9_corridor_metrics_speeds.geojson')
second_set['submission_round'] = 2

In [35]:
access_results = pd.read_csv('./AccessibilityResults.csv')

In [36]:
first_set >> head(3)

,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,corridor,geometry,submission_round
0,28,28,City and County of San Francisco,9.3,11.8,13.2,480.1,1690.8,56.4,159,14.9,5.3,8.5,5.6,1.1,1739.0,16,us101_lombard,"POLYGON ((-214489.679 -21257.665, -214484.673 ...",1
1,43,43,City and County of San Francisco,8.1,9.5,10.1,1210.7,1690.8,73.2,143,14.8,4.8,16.5,5.4,1.1,NaN,16,us101_lombard,"POLYGON ((-214489.679 -21257.665, -214484.673 ...",1
2,28,28,City and County of San Francisco,9.6,11.5,11.9,1061.7,1347.9,56.4,159,14.9,5.3,18.8,6.0,2.4,3850.5,16,sr1_19th_a,"POLYGON ((-217949.193 -26918.889, -217955.214 ...",1


In [37]:
second_set >> head(3)

,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,corridor,geometry,submission_round
0,4-13168,4,Los Angeles County Metropolitan Transportation...,4.7,11.1,9.3,94.2,94.2,96.2,33,2.3,7.2,1.0,7.4,1.2,1982.4,16,sr2_am,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2
1,4-13168,4,Los Angeles County Metropolitan Transportation...,4.9,6.8,7.9,342.4,342.4,153.5,42,3.1,6.7,2.2,7.1,1.3,2105.6,16,sr2_pm,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2
2,4-13168,4,Los Angeles County Metropolitan Transportation...,3.4,8.8,7.8,685.4,685.4,131.2,111,6.9,8.0,5.2,8.4,1.3,2105.6,16,sr2_mid,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2


In [38]:
all_corridors = pd.concat([first_set, second_set])

In [39]:
all_grouped = (all_corridors >> group_by(_.corridor, _.organization, _.submission_round)  # length and span cause fan-out, re-add later...
               >> summarize(sum_trips =_.n_trips.sum(), total_delay = _.total_speed_delay.max())
               >> mutate(avg_delay_minutes = _.total_delay / _.sum_trips)
)

## note: don't sum/aggregate accessibility results within a corridor

In [61]:
rider_access_joined = all_grouped >> left_join(_, access_results, on=['corridor', 'submission_round'])

# Adding ridership (added to accessibility csv)

## apportioning ridership for partial-hour bus lane on sr2

* evenly during daytime hours (0500 - 2200), match post-covid trends of less peaking
* 22,669 daily for entire corr
* 

## notes
* for SDMTS, sum ridership from routes 60, 235, 225

In [41]:
metro_4_ridership = 22669
riders_per_hour = metro_4_ridership / (22 - 5)

In [60]:
rider_estimates = (second_set >> filter(_.corridor.str.contains('sr2'))
 >> select(_.corridor, _.span_hours, _.submission_round)
 >> mutate(daily_riders = _.span_hours * riders_per_hour)
 >> select(-_.span_hours)
)
rider_estimates

,corridor,submission_round,daily_riders
0,sr2_am,2,3066.982353
1,sr2_pm,2,4133.758824
2,sr2_mid,2,9200.947059
3,sr2_eve,2,3467.023529
4,sr2_early_am,2,1333.470588


In [62]:
rider_access_joined = rider_access_joined >> left_join(_, rider_estimates, on =['corridor', 'submission_round'])

In [63]:
rider_access_joined['daily_riders'] = rider_access_joined.daily_riders_x.fillna(rider_access_joined.daily_riders_y)

In [64]:
rider_access_joined = rider_access_joined >> select(-_.daily_riders_x, -_.daily_riders_y)

# Testing metrics

# Create defined projects from disaggregated sr2

In [68]:
df = rider_access_joined >> filter(_.corridor.str.contains('sr2'), _.submission_round == 2)

In [69]:
df

,corridor,organization,submission_round,sum_trips,total_delay,avg_delay_minutes,avg_pct_chng_access,pct_change_access_Jobs,pct_change_access_weighted_Jobs,avg_change_access_Jobs,weighted_avg_change_access_Jobs,pct_change_access_POIs,pct_change_access_weighted_POIs,avg_change_access_POIs,weighted_avg_change_access_POIs,pct_li_sub50k,daily_riders
7,sr2_am,Los Angeles County Metropolitan Transportation...,2,33,94.2,2.854545,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,0.83,3066.982353
8,sr2_early_am,Los Angeles County Metropolitan Transportation...,2,14,4.1,0.292857,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,0.83,1333.470588
9,sr2_eve,Los Angeles County Metropolitan Transportation...,2,23,89.8,3.904348,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,0.83,3467.023529
10,sr2_mid,Los Angeles County Metropolitan Transportation...,2,111,685.4,6.174775,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,0.83,9200.947059
11,sr2_pm,Los Angeles County Metropolitan Transportation...,2,42,342.4,8.152381,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,0.83,4133.758824


In [87]:
df2 = df[['sum_trips', 'total_delay', 'daily_riders']].sum()
df2['avg_delay_minutes'] = df2.total_delay / df2.sum_trips
df2['corridor'] = 'sr2_all_day'
df2_dict = df2.to_dict()

In [88]:
unchanged_series = (df >> select(-_.sum_trips, -_.total_delay, -_.daily_riders, -_.avg_delay_minutes, -_.corridor)).iloc[0,:]

In [89]:
unchanged_dict = unchanged_series.to_dict()

In [90]:
merged_dict = df2_dict | unchanged_dict

In [91]:
merged_dict

{'sum_trips': 223.0,
 'total_delay': 1215.9,
 'daily_riders': 21202.182352941178,
 'avg_delay_minutes': 5.452466367713005,
 'corridor': 'sr2_all_day',
 'organization': 'Los Angeles County Metropolitan Transportation Authority',
 'submission_round': 2,
 'avg_pct_chng_access': 0.001649968,
 'pct_change_access_Jobs': 0.0016262,
 'pct_change_access_weighted_Jobs': 0.001700249,
 'avg_change_access_Jobs': 195.9254735,
 'weighted_avg_change_access_Jobs': 586.8094511,
 'pct_change_access_POIs': 0.001733737,
 'pct_change_access_weighted_POIs': 0.001599687,
 'avg_change_access_POIs': 2.475954141,
 'weighted_avg_change_access_POIs': 6.63503573,
 'pct_li_sub50k': 0.83}

In [94]:
all_day_df = pd.DataFrame(merged_dict, index=[99])

In [96]:
with_sr2 = pd.concat([rider_access_joined, all_day_df]).reset_index(drop=True)

In [97]:
with_sr2

,corridor,organization,submission_round,sum_trips,total_delay,avg_delay_minutes,avg_pct_chng_access,pct_change_access_Jobs,pct_change_access_weighted_Jobs,avg_change_access_Jobs,weighted_avg_change_access_Jobs,pct_change_access_POIs,pct_change_access_weighted_POIs,avg_change_access_POIs,weighted_avg_change_access_POIs,pct_li_sub50k,daily_riders
0,i805,San Diego Metropolitan Transit System,2,90.0,99.5,1.105556,0.015327,0.015211,0.017178,216.299637,1308.461745,0.012527,0.013475,2.518450,14.145989,0.79,4233.000000
1,sr123,Alameda-Contra Costa Transit District,2,219.0,872.0,3.981735,0.001397,0.001457,0.001271,75.015147,338.098215,0.001654,0.001523,0.953931,4.389842,0.65,14789.000000
2,sr123_san_pablo,Alameda-Contra Costa Transit District,1,250.0,3623.2,14.492800,0.003879,0.003989,0.003270,209.573414,905.891038,0.005166,0.004487,3.029337,13.415441,0.65,14789.000000
3,sr15,San Diego Metropolitan Transit System,2,109.0,313.9,2.879817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sr1_19th_a,City and County of San Francisco,1,339.0,1347.9,3.976106,0.003184,0.002458,0.002352,119.881855,733.304443,0.004557,0.004016,2.486017,13.624096,0.70,12100.000000
5,sr1_19th_b,City and County of San Francisco,1,339.0,956.6,2.821829,0.003184,0.002458,0.002352,119.881855,733.304443,0.004557,0.004016,2.486017,13.624096,0.70,12100.000000
6,sr1_lincoln,City of Santa Monica,1,115.0,925.9,8.051304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,sr2_am,Los Angeles County Metropolitan Transportation...,2,33.0,94.2,2.854545,0.001650,0.001626,0.001700,195.925474,586.809451,0.001734,0.001600,2.475954,6.635036,0.83,3066.982353
8,sr2_early_am,Los Angeles County Metropolitan Transportation...,2,14.0,4.1,0.292857,0.001650,0.001626,0.001700,195.925474,586.809451,0.001734,0.001600,2.475954,6.635036,0.83,1333.470588
9,sr2_eve,Los Angeles County Metropolitan Transportation...,2,23.0,89.8,3.904348,0.001650,0.001626,0.001700,195.925474,586.809451,0.001734,0.001600,2.475954,6.635036,0.83,3467.023529
